**Connect google drive**

In [29]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [8]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s  /content/drive/My\ Drive/ /mydrive  # !ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive': File exists
/mydrive


**1) Clone, configure & compile Darknet**



In [2]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 14751 (delta 0), reused 2 (delta 0), pack-reused 14748
Receiving objects: 100% (14751/14751), 13.28 MiB | 13.33 MiB/s, done.
Resolving deltas: 100% (10029/10029), done.


In [3]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [4]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

**2) Configure yolov3.cfg file**

In [15]:
# Make a copy of yolov3.cfg or yolov3-tiny.cfg
!cp cfg/yolov3-tiny.cfg cfg/yolov3-tiny_training.cfg
#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [10]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3-tiny_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3-tiny_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3-tiny_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3-tiny_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3-tiny_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3-tiny_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3-tiny_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3-tiny_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3-tiny_training.cfg

**3) Create .names and .data files**

In [12]:
!echo -e 'researcher\nrat\nhead' > /content/darknet/data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = content/gdrive/MyDrive/yolov3' > /content/darknet/data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [18]:
##Already saved in MyDrive/yolov3
!cp /content/darknet/cfg/yolov3-tiny_training.cfg /content/gdrive/MyDrive/yolov3/yolov3-tiny_testing.cfg #yolov3_training.cfg 
#!cp /content/darknet/data/obj.names /content/gdrive/MyDrive/yolov3/classes.txt

**5) Create a folder and unzip image dataset**

In [19]:
!mkdir /content/darknet/data/obj
!unzip /content/gdrive/MyDrive/yolov3/TrainDataset.zip -d /content/darknet/data/obj

Archive:  /content/gdrive/MyDrive/yolov3/TrainDataset.zip
  inflating: /content/darknet/data/obj/image_000000.jpeg  
  inflating: /content/darknet/data/obj/image_000000.txt  
  inflating: /content/darknet/data/obj/image_000001.jpeg  
  inflating: /content/darknet/data/obj/image_000001.txt  
  inflating: /content/darknet/data/obj/image_000002.jpeg  
  inflating: /content/darknet/data/obj/image_000002.txt  
  inflating: /content/darknet/data/obj/image_000003.jpeg  
  inflating: /content/darknet/data/obj/image_000003.txt  
  inflating: /content/darknet/data/obj/image_000004.jpeg  
  inflating: /content/darknet/data/obj/image_000004.txt  
  inflating: /content/darknet/data/obj/image_000005.jpeg  
  inflating: /content/darknet/data/obj/image_000005.txt  
  inflating: /content/darknet/data/obj/image_000006.jpeg  
  inflating: /content/darknet/data/obj/image_000006.txt  
  inflating: /content/darknet/data/obj/image_000007.jpeg  
  inflating: /content/darknet/data/obj/image_000007.txt  
  infl

**6) Create train.txt file**

In [22]:
import glob
images_list = glob.glob("data/obj/*.jpeg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [27]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-04-12 22:56:52--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M  97.1MB/s    in 1.6s    

2021-04-12 22:56:54 (97.1 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



**8) Start training**

In [28]:
##CHECK darknet/data/obj.data FILE - backub=  /content/drive/MyDrive/yolov3 
!./darknet detector train data/obj.data cfg/yolov3-tiny_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /content/gdrive/MyDrive/yolov3 -dont_show
##run this cell to keep runtime connected 


CUDA status Error: file: ./src/dark_cuda.c : () : line: 39 : build time: Apr 12 2021 - 22:24:41 

 CUDA Error: no CUDA-capable device is detected
CUDA Error: no CUDA-capable device is detected: Bad file descriptor
darknet: ./src/utils.c:331: error: Assertion `0' failed.


In [ ]:
##run this cell to keep runtime connected 
while True:pass